In [1]:
import sys
sys.path.insert(0, '../src') # to be able to import functions from src

import torch
import argparse
import os
import logging as log
from unet import *
from noise_scheduler import *
from sampler import *
from ddpm_sampler import *
from ddim_sampler import *

# network hyperparameters
n_feat = 64 # 64 hidden dimension feature
n_cfeat = 5 # context vector is of size 5
height = 16 # 16x16 image
in_channels=3 # rgb

model_filename = f"../weights/model_trained_500_LINEAR_100_32_context.pth"
nn_model = ContextUnet(in_channels, n_feat=n_feat, n_cfeat=n_cfeat, height=height)
nn_model.load_state_dict(torch.load(model_filename))
nn_model.eval()

#nn_model.train()
#torch.save(nn_model, "model.pth")
#m = torch.load("model.pth")
#m.eval()

C:\Users\jgregoire\AppData\Local\Temp\ipykernel_32336\718647636.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  nn_model.load_state_dict(torch.load(model_filename))


RuntimeError: Error(s) in loading state_dict for ContextUnet:
	Missing key(s) in state_dict: "init_conv.shortcut.weight", "init_conv.shortcut.bias". 

In [2]:
from onnx import __version__, IR_VERSION
from onnx.defs import onnx_opset_version
print(f"onnx.__version__={__version__!r}, opset={onnx_opset_version()}, IR_VERSION={IR_VERSION}")

onnx.__version__='1.16.2', opset=21, IR_VERSION=10


In [3]:
samples = torch.randn(9, 3, height, height) 
t = torch.tensor([1 / 500])[:, None, None, None]
eps = nn_model(samples, t)
onnx_program = torch.onnx.dynamo_export(nn_model, (samples, t))

d:\bin\miniforge3\envs\pytorch2\Lib\site-packages\torch\onnx\_internal\exporter.py:137: UserWarning: torch.onnx.dynamo_export only implements opset version 18 for now. If you need to use a different opset version, please register them with register_custom_op.
  warnings.warn(


OnnxExporterError: Failed to export the model to ONNX. Generating SARIF report at 'report_dynamo_export.sarif'. SARIF is a standard format for the output of static analysis tools. SARIF logs can be loaded in VS Code SARIF viewer extension, or SARIF web viewer (https://microsoft.github.io/sarif-web-component/). Please report a bug on PyTorch Github: https://github.com/pytorch/pytorch/issues

In [ ]:
# Export the model   
torch.onnx.export(nn_model,         # model being run 
         (samples, t),       # model input (or a tuple for multiple inputs) 
         "mymodel.onnx",       # where to save the model  
         export_params=True,  # store the trained parameter weights inside the model file 
         opset_version=10,    # the ONNX version to export the model to 
         do_constant_folding=True,  # whether to execute constant folding for optimization 
         input_names = ['samples', 'timestamp'],   # the model's input names 
         output_names = ['modelOutput'], # the model's output names 
) 

UnsupportedOperatorError: Exporting the operator 'aten::uniform' to ONNX opset version 10 is not supported. Please feel free to request support or submit a pull request on PyTorch GitHub: https://github.com/pytorch/pytorch/issues.

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
import torch.onnx

# Define a simple CNN
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout2d(0.25)
        self.dropout2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = nn.functional.relu(x)
        x = self.conv2(x)
        x = nn.functional.relu(x)
        x = nn.functional.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = nn.functional.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = nn.functional.log_softmax(x, dim=1)
        return output

# Initialize the model
model = SimpleCNN()

# Define a dummy input (adjust the size according to your model's input requirements)
dummy_input = torch.randn(1, 1, 28, 28)

# Export the model to ONNX
torch.onnx.export(model,               # model being run
                  dummy_input,         # model input (or a tuple for multiple inputs)
                  "simple_cnn.onnx",   # where to save the model
                  export_params=True,  # store the trained parameter weights inside the model file
                  opset_version=10,    # the ONNX version to export the model to
                  do_constant_folding=True,  # whether to execute constant folding for optimization
                  input_names = ['input'],   # the model's input names
                  output_names = ['output'])

print("Model exported to simple_cnn.onnx")

# Optionally, you can verify the model export:
import onnx

# Load the ONNX model
onnx_model = onnx.load("simple_cnn.onnx")

# Check that the model is well formed
onnx.checker.check_model(onnx_model)

print("ONNX model is well formed")


onnx_program = torch.onnx.dynamo_export(model, dummy_input)
onnx_program.save("simple_cnn2.onnx")

d:\bin\miniforge3\envs\pytorch2\Lib\site-packages\torch\nn\functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)


Model exported to simple_cnn.onnx
ONNX model is well formed


d:\bin\miniforge3\envs\pytorch2\Lib\site-packages\torch\nn\functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
d:\bin\miniforge3\envs\pytorch2\Lib\site-packages\torch\nn\functional.py:1374: UserWarning: dropout2d: Received a 2-D input to dropout2d, which is deprecated and will result in an error in a future release. To retain the behavior and silence this warning, please use dropout instead. Note that dropout2d exists to provide channel-wise dropout on inputs with 2 spatial dimensions, a channel dimension, and an optional batch dimension (i.e. 3D or 4D inputs).
  warnings.warn(warn_msg)
d:\bin\miniforge3\envs\pytorch2\Li

In [25]:
m = ContextUnet(in_channels, n_feat=n_feat, n_cfeat=n_cfeat, height=height)
samples = torch.randn(9, 3, height, height) 
t = torch.tensor([1 / 500])[:, None, None, None]
torch.onnx.export(m,         # model being run 
         (samples, t),       # model input (or a tuple for multiple inputs) 
         "mymodel.onnx",       # where to save the model  
         export_params=True,  # store the trained parameter weights inside the model file 
         opset_version=10,    # the ONNX version to export the model to 
         do_constant_folding=True,  # whether to execute constant folding for optimization 
         input_names = ['samples', 'timestamp'],   # the model's input names 
         output_names = ['modelOutput'], # the model's output names 
) 

In [5]:
import torch
import torch.onnx
from typing import Optional

# First, let's copy the necessary class definitions from the provided code
# (ResidualConvBlock, UnetUp, UnetDown, EmbedFC, ContextUnet)

# ... [Copy all the class definitions here] ...

# Now let's create a script to convert the model to ONNX

def convert_to_onnx(model: ContextUnet, 
                    save_path: str, 
                    input_shape: tuple = (1, 3, 28, 28),
                    time_shape: tuple = (1, 1),
                    context_shape: Optional[tuple] = None):
    
    # Create dummy inputs
    dummy_input = torch.randn(input_shape)
    dummy_time = torch.randn(time_shape)
    
    if context_shape is not None:
        dummy_context = torch.randn(context_shape)
    else:
        dummy_context = None

    # Set the model to evaluation mode
    model.eval()

    # Trace the model
    traced_model = torch.jit.trace(model, (dummy_input, dummy_time, dummy_context))

    # Export the model
    torch.onnx.export(model,
                      (dummy_input, dummy_time, dummy_context),
                      save_path,
                      export_params=True,
                      opset_version=11,
                      do_constant_folding=True,
                      input_names=['input', 'time', 'context'],
                      output_names=['output'],
                      dynamic_axes={'input': {0: 'batch_size'},
                                    'time': {0: 'batch_size'},
                                    'context': {0: 'batch_size'},
                                    'output': {0: 'batch_size'}})

    print(f"Model has been converted to ONNX and saved at {save_path}")
    #onnx_program = torch.onnx.dynamo_export(model, (dummy_input, dummy_time, dummy_context))


torch.manual_seed(0)
model = ContextUnet(in_channels=3, n_feat=256, n_cfeat=10, height=28)
    
    # Convert to ONNX
convert_to_onnx(model, 
                    save_path="context_unet.onnx", 
                    input_shape=(1, 3, 28, 28),
                    time_shape=(1, 1),
                    context_shape=(1, 10))

Model has been converted to ONNX and saved at context_unet.onnx


In [24]:
import torch
import torch.onnx
from typing import Optional

# [Include all the class definitions here as before]

class ContextUnetWrapper(torch.nn.Module):
    def __init__(self, model):
        super().__init__()
        self.model = model

    def forward(self, x, t, c):
        return self.model(x, t, c)

def convert_to_onnx(model: ContextUnet, 
                    save_path: str, 
                    input_shape: tuple = (1, 3, 28, 28),
                    time_shape: tuple = (1, 1),
                    context_shape: Optional[tuple] = None):
    
    # Create dummy inputs
    dummy_input = torch.randn(input_shape)
    dummy_time = torch.randn(time_shape)
    
    if context_shape is not None:
        dummy_context = torch.randn(context_shape)
    else:
        dummy_context = None

    # Set the model to evaluation mode
    model.eval()

    # Wrap the model
    wrapped_model = ContextUnetWrapper(model)

    # Script the model
    scripted_model = wrapped_model

    # Debug: Compare outputs
    with torch.no_grad():
        python_output = model(dummy_input, dummy_time, dummy_context)
        script_output = scripted_model(dummy_input, dummy_time, dummy_context)
    
    print("Python output shape:", python_output.shape)
    print("Scripted output shape:", script_output.shape)
    print("Output difference:", torch.max(torch.abs(python_output - script_output)))

    # Export the scripted model
    torch.onnx.export(scripted_model,
                      (dummy_input, dummy_time, dummy_context),
                      save_path,
                      export_params=True,
                      opset_version=12,
                      do_constant_folding=True,
                      input_names=['input', 'time', 'context'],
                      output_names=['output'],
                      dynamic_axes={'input': {0: 'batch_size'},
                                    'time': {0: 'batch_size'},
                                    'context': {0: 'batch_size'},
                                    'output': {0: 'batch_size'}})
                                    

    print(f"Model has been converted to ONNX and saved at {save_path}")

# Usage
if __name__ == "__main__":
    # Set a fixed seed for reproducibility
    torch.manual_seed(0)
    
    # Initialize the model
    model = ContextUnet(in_channels=3, n_feat=256, n_cfeat=10, height=28)
    
    # Convert to ONNX
    convert_to_onnx(model, 
                    save_path="context_unet.onnx", 
                    input_shape=(1, 3, 28, 28),
                    time_shape=(1, 1),
                    context_shape=(1, 10))
    
    

Python output shape: torch.Size([1, 3, 28, 28])
Scripted output shape: torch.Size([1, 3, 28, 28])
Output difference: tensor(0.)
Model has been converted to ONNX and saved at context_unet.onnx


In [10]:
# Define input tensor with shape (batch_size, in_channels, height, width)
batch_size = 2
in_channels = 3
out_channels = 3
height = 64
width = 64

dummy_input = torch.randn(batch_size, in_channels, height, width)

# Instantiate the block with and without residual connections
block_res = ResidualConvBlock(in_channels, out_channels, is_res=True)
block_no_res = ResidualConvBlock(in_channels, out_channels, is_res=False)

# Pass the input tensor through the blocks
output_res = block_res(dummy_input)
output_no_res = block_no_res(dummy_input)

# Print the output shapes
print(f"Output shape with residual connection: {output_res.shape}")
print(f"Output shape without residual connection: {output_no_res.shape}")

torch.onnx.export(block_res,
                    dummy_input,
                    "context_unet.onnx",
                    export_params=True,
                    opset_version=12,
                    do_constant_folding=True,
                    input_names=['input', 'time', 'context'],
                    output_names=['output'],
                    dynamic_axes={'input': {0: 'batch_size'},
                                'time': {0: 'batch_size'},
                                'context': {0: 'batch_size'},
                                'output': {0: 'batch_size'}})

import onnx
onnx_model = onnx.load("context_unet.onnx")
onnx.checker.check_model(onnx_model)

Output shape with residual connection: torch.Size([2, 3, 64, 64])
Output shape without residual connection: torch.Size([2, 3, 64, 64])


In [ ]:
# Define input tensors with appropriate shapes
batch_size = 2
in_channels = 128
out_channels = 64
h, w = 32, 32  # spatial dimensions of the input
h_skip, w_skip = 32, 32  # spatial dimensions of the skip connection

# Create dummy input tensors
x = torch.randn(batch_size, in_channels, h, w)  # input tensor
skip = torch.randn(batch_size, out_channels, h_skip, w_skip)  # skip connection tensor

# Instantiate the UnetUp block
up_block = UnetUp(in_channels + out_channels, out_channels)

# Pass the tensors through the block
output = up_block(x, skip)

# Print the output shape
print(f"Output shape: {output.shape}")

torch.onnx.export(up_block,
                    (x, skip),
                    "context_unet.onnx",
                    export_params=True,
                    opset_version=12,
                    do_constant_folding=True,
                    input_names=['input', 'time', 'context'],
                    output_names=['output'],
                    dynamic_axes={'input': {0: 'batch_size'},
                                'time': {0: 'batch_size'},
                                'context': {0: 'batch_size'},
                                'output': {0: 'batch_size'}})

import onnx
onnx_model = onnx.load("context_unet.onnx")
onnx.checker.check_model(onnx_model)

Output shape: torch.Size([2, 64, 64, 64])


In [7]:
# Define input tensors with appropriate shapes
batch_size = 2
in_channels = 128
out_channels = 64
h, w = 32, 32  # spatial dimensions of the input

# Create a dummy input tensor
x = torch.randn(batch_size, in_channels, h, w)

# Instantiate the UnetDown block
down_block = UnetDown(in_channels, out_channels)

# Pass the tensor through the block
output = down_block(x)

# Print the output shape
print(f"Output shape: {output.shape}")

torch.onnx.export(down_block,
                    x,
                    "context_unet.onnx",
                    export_params=True,
                    opset_version=12,
                    do_constant_folding=True,
                    input_names=['input', 'time', 'context'],
                    output_names=['output'],
                    dynamic_axes={'input': {0: 'batch_size'},
                                'time': {0: 'batch_size'},
                                'context': {0: 'batch_size'},
                                'output': {0: 'batch_size'}})

import onnx
onnx_model = onnx.load("context_unet.onnx")
onnx.checker.check_model(onnx_model)

Output shape: torch.Size([2, 64, 16, 16])


In [8]:
# Define input tensor with appropriate shape
batch_size = 4
input_dim = 10  # dimensionality of the input
emb_dim = 20    # dimensionality of the embedding space

# Create a dummy input tensor
x = torch.randn(batch_size, input_dim)

# Instantiate the EmbedFC block
embed_fc = EmbedFC(input_dim, emb_dim)

# Pass the tensor through the block
output = embed_fc(x)

# Print the output shape
print(f"Output shape: {output.shape}")


torch.onnx.export(embed_fc,
                    x,
                    "context_unet.onnx",
                    export_params=True,
                    opset_version=12,
                    do_constant_folding=True,
                    input_names=['input', 'time', 'context'],
                    output_names=['output'],
                    dynamic_axes={'input': {0: 'batch_size'},
                                'time': {0: 'batch_size'},
                                'context': {0: 'batch_size'},
                                'output': {0: 'batch_size'}})

import onnx
onnx_model = onnx.load("context_unet.onnx")
onnx.checker.check_model(onnx_model)

Output shape: torch.Size([4, 20])


In [23]:
class tmpContextUnet(nn.Module):
    def __init__(self, in_channels, n_feat=256, n_cfeat=10, height=28):  # cfeat - context features
        super(tmpContextUnet, self).__init__()

        # number of input channels, number of intermediate feature maps and number of classes
        self.in_channels = in_channels
        self.n_feat = n_feat
        self.n_cfeat = n_cfeat
        self.h = height  #assume h == w. must be divisible by 4, so 28,24,20,16...

        # Initialize the initial convolutional layer
        self.init_conv = ResidualConvBlock(in_channels, n_feat, is_res=True)

        # Initialize the down-sampling path of the U-Net with two levels
        self.down1 = UnetDown(n_feat, n_feat)        # down1 #[10, 256, 8, 8]
        self.down2 = UnetDown(n_feat, 2 * n_feat)    # down2 #[10, 256, 4,  4]
        
         # original: self.to_vec = nn.Sequential(nn.AvgPool2d(7), nn.GELU())
        self.to_vec = nn.Sequential(nn.AvgPool2d((4)), nn.GELU())

        # Embed the timestep and context labels with a one-layer fully connected neural network
        self.timeembed1 = EmbedFC(1, 2*n_feat)
        self.timeembed2 = EmbedFC(1, 1*n_feat)
        self.contextembed1 = EmbedFC(n_cfeat, 2*n_feat)
        self.contextembed2 = EmbedFC(n_cfeat, 1*n_feat)

        # Initialize the up-sampling path of the U-Net with three levels
        self.up0 = nn.Sequential(
            nn.ConvTranspose2d(2 * n_feat, 2 * n_feat, self.h//4, self.h//4), # up-sample  
            nn.GroupNorm(8, 2 * n_feat), # normalize                       
            nn.ReLU(),
        )
        self.up1 = UnetUp(4 * n_feat, n_feat)
        self.up2 = UnetUp(2 * n_feat, n_feat)

        # Initialize the final convolutional layers to map to the same number of channels as the input image
        self.out = nn.Sequential(
            nn.Conv2d(2 * n_feat, n_feat, 3, 1, 1), # reduce number of feature maps   #in_channels, out_channels, kernel_size, stride=1, padding=0
            nn.GroupNorm(8, n_feat), # normalize
            nn.ReLU(),
            nn.Conv2d(n_feat, self.in_channels, 3, 1, 1), # map to same number of channels as input
        )

    def forward(self, x, t, c=None):
        """
        x : (batch, n_feat, h, w) : input image
        t : (batch, n_cfeat)      : time step
        c : (batch, n_classes)    : context label
        """
        # x is the input image, c is the context label, t is the timestep, context_mask says which samples to block the context on

        # pass the input image through the initial convolutional layer
        x = self.init_conv(x)
        # pass the result through the down-sampling path
        down1 = self.down1(x)       #[10, 256, 8, 8]
        down2 = self.down2(down1)   #[10, 256, 4, 4]
        
        # convert the feature maps to a vector and apply an activation
        hiddenvec = self.to_vec(down2)
        
        # mask out context if context_mask == 1
        if c is None:
            c = torch.zeros(x.shape[0], self.n_cfeat).to(x)
            
        # embed context and timestep
        cemb1 = self.contextembed1(c).view(-1, self.n_feat * 2, 1, 1)     # (batch, 2*n_feat, 1,1)
        temb1 = self.timeembed1(t).view(-1, self.n_feat * 2, 1, 1)
        cemb2 = self.contextembed2(c).view(-1, self.n_feat, 1, 1)
        temb2 = self.timeembed2(t).view(-1, self.n_feat, 1, 1)
        log.debug(f"uunet forward: cemb1 {cemb1.shape}. temb1 {temb1.shape}, cemb2 {cemb2.shape}. temb2 {temb2.shape}")


        up1 = self.up0(hiddenvec)
        up2 = self.up1(cemb1*up1 + temb1, down2)  # add and multiply embeddings
        up3 = self.up2(cemb2*up2 + temb2, down1)
        out = self.out(torch.cat((up3, x), 1))
        return out


# Define input parameters
batch_size = 4
in_channels = 3       # Number of input channels (e.g., RGB image)
height = 28           # Height and width of the input image (assumed square)
n_feat = 256          # Number of intermediate feature maps
n_cfeat = 10          # Number of context features

# Create dummy input tensors
x = torch.randn(batch_size, in_channels, height, height)  # Input image tensor
t = torch.randn(batch_size, 1)                            # Time step tensor
c = torch.randn(batch_size, n_cfeat)                      # Context tensor

# Instantiate the ContextUnet model
context_unet = ContextUnet(in_channels, n_feat=n_feat, n_cfeat=n_cfeat, height=height)

# Pass the tensors through the model
output = context_unet(x,t,c)

# Print the output shape
print(f"Output shape: {output.shape}")

torch.onnx.export(context_unet,
                    (x,t,c),
                    "context_unet.onnx",
                    export_params=True,
                    opset_version=12,
                    do_constant_folding=True,
                    input_names=['input', 'time', 'context'],
                    output_names=['output'],
                    dynamic_axes={'input': {0: 'batch_size'},
                                'time': {0: 'batch_size'},
                                'context': {0: 'batch_size'},
                                'output': {0: 'batch_size'}})

import onnx
onnx_model = onnx.load("context_unet.onnx")
onnx.checker.check_model(onnx_model)

Output shape: torch.Size([4, 3, 28, 28])
